# mapping custom tags 

In [1]:
import pandas as pd
import numpy as np

In [2]:
import plac
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc, Span, Token


In [3]:
def load_drug_entities():
    drugs = np.load("/Users/sdeshpande/Desktop/bioinformatices/Clinical_NER_train/drug_entities.npy")
    drugs = [d.lower() for d in drugs if (len(d)>4)]
    non_ents = ["solution"]
    for ent in non_ents:
        drugs.remove(ent)
    drugs = list(set(drugs))
    return drugs

In [4]:
drugs = load_drug_entities()

In [5]:
drugs[0:20]

['etravirine',
 '*nf* etodolac',
 'pentasa',
 'zenpep',
 'nitrofurantoin macrocrystal',
 'detrol la',
 'docusate sodiu',
 'beclomethasone dipro. aq (nasal)',
 'prednisolone acetate 0.12% ophth. susp.',
 'vaseline lip therapy',
 'lytes/carboxymethylcellulose',
 'symlin',
 'repaglinide',
 'etomidate',
 'nicot',
 'dextromethorphan-guaifenesin (sugar free)',
 'crestor',
 'neurontin',
 '<ind>eptifibatide study drug',
 'niaspan extended-release']

In [6]:
def load_drug_entities():
    drugs = np.load("/Users/sdeshpande/Desktop/bioinformatices/Clinical_NER_train/drug_entities.npy")
    drugs = [d.lower() for d in drugs if (len(d)>4)]
    non_ents = ["solution"]
    for ent in non_ents:
        drugs.remove(ent)
    drugs = list(set(drugs))
    return drugs

In [7]:
def load_dose_entities():
    doses = np.load("/Users/sdeshpande/Desktop/bioinformatices/Clinical_NER_train/dose_entities.npy")
    doses = [d.lower() for d in doses if len(d)>1]
    return list(set(doses))

In [8]:
dose_ents = load_dose_entities()
print(dose_ents[:10])

['0.1 mg/hr patch (2.5mg)', '100 % topical spray', '80mg/1ml vial', '650mg supp', '500mg cap', '5mg/ml oral solution', '0.01 % eye drops', '100 mg capsule', '500mg/50ml vial', '60mg/ml; 120ml bottle']


In [9]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [10]:
import spacy

In [22]:
nlp = spacy.load('en_core_web_md')

In [13]:
class EntityMatcher(object):
    name = 'entity_matcher'

    def __init__(self, nlp, drugs, label):
        patterns = [nlp(text) for text in drugs]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add(label, None, *patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        seen_tokens = set()
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
        #    span = Span(doc, start, end, label=match_id)
        #    doc.ents = list(doc.ents) + [span]
            # check for end - 1 here because boundaries are inclusive
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc

In [14]:
class EntityMatcherDose(object):
    name = 'entity_matcher_dose'

    def __init__(self, nlp, dose_ents, label):
        patterns = [nlp(text) for text in dose_ents]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add(label, None, *patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        seen_tokens = set()
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
        #    span = Span(doc, start, end, label=match_id)
        #    doc.ents = list(doc.ents) + [span]
            # check for end - 1 here because boundaries are inclusive
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc

In [15]:
from spacy.matcher import Matcher

In [16]:
entity_matcher_3 = Matcher(nlp.vocab)
pattern = [{"TEXT": {"REGEX": "[0-9]+"}}]
entity_matcher_3.add("MEASUREMENTS", None, pattern)

In [35]:
class EntityMatcherNumber(object):
    name = 'entity_matcher_number'

    def __init__(self, nlp, label):
        patterns = [{"TEXT": {"REGEX": "[0-9]+"}}]
        self.matcher = Matcher(nlp.vocab)
        self.matcher.add(label, None, patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        seen_tokens = set()
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
        #    span = Span(doc, start, end, label=match_id)
        #    doc.ents = list(doc.ents) + [span]
            # check for end - 1 here because boundaries are inclusive
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc

In [36]:
entity_matcher_3 = EntityMatcherNumber(nlp, 'MEASUREMENTS')

In [17]:
### Regex pattern matching reference code for spacy

# nlp = spacy.load("en_core_web_lg")
# matcher = Matcher(nlp.vocab)
# pattern = [{"TEXT": {"REGEX": "^[Uu](\.?|nited)$"}},
#            {"TEXT": {"REGEX": "^[Ss](\.?|tates)$"}}]
# matcher.add("US", None, pattern)
# doc = nlp(u"I'm from the United States.")
# matches = matcher(doc)
# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]
#     span = doc[start:end]
#     print(match_id, string_id, start, end, span.text)

In [23]:
entity_matcher = EntityMatcher(nlp, drugs, 'DRUGS')
entity_matcher_2 = EntityMatcherDose(nlp, dose_ents, 'DOSES')

In [24]:
nlp.add_pipe(entity_matcher)
nlp.add_pipe(entity_matcher_2)
print(nlp.pipe_names)  # the components in the pipeline

['tagger', 'parser', 'ner', 'entity_matcher', 'entity_matcher_dose']


In [37]:
nlp.add_pipe(entity_matcher_3)
print(nlp.pipe_names)

['tagger', 'parser', 'ner', 'entity_matcher', 'entity_matcher_dose', 'entity_matcher_number']


In [38]:
doc = nlp(u"guanfacine, capecitabine, ertapenem sodium, insulin lispro, isosorbide mononitrate , nexium, aprotinin, metronidazole (flagyl), critic-aid, 0.4mg/2ml vial, 0.4mg/hr patch (10mg), 50 mg capsule")
print([(ent.text, ent.label_) for ent in doc.ents])

[('guanfacine', 'DRUGS'), ('ertapenem', 'DRUGS'), ('sodium', 'DRUGS'), ('insulin', 'DRUGS'), ('isosorbide', 'DRUGS'), ('nexium', 'DRUGS'), ('aprotinin', 'DRUGS'), ('metronidazole', 'DRUGS'), ('critic-aid', 'DRUGS'), ('0.4mg/2ml', 'MEASUREMENTS'), ('0.4mg', 'MEASUREMENTS'), ('10', 'MEASUREMENTS'), ('50', 'MEASUREMENTS'), ('mg', 'DOSES'), ('capsule', 'DOSES')]


In [39]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

guanfacine 0 10 DRUGS
ertapenem 26 35 DRUGS
sodium 36 42 DRUGS
insulin 44 51 DRUGS
isosorbide 60 70 DRUGS
nexium 85 91 DRUGS
aprotinin 93 102 DRUGS
metronidazole 104 117 DRUGS
critic-aid 128 138 DRUGS
0.4mg/2ml 140 149 MEASUREMENTS
0.4mg 156 161 MEASUREMENTS
10 172 174 MEASUREMENTS
50 179 181 MEASUREMENTS
mg 182 184 DOSES
capsule 185 192 DOSES


In [40]:
pd.DataFrame([(e.text, e.ent_iob_, e.ent_type_) for e in doc])

,0,1,2
0,guanfacine,B,DRUGS
1,",",O,
2,capecitabine,O,
3,",",O,
4,ertapenem,B,DRUGS
5,sodium,B,DRUGS
6,",",O,
7,insulin,B,DRUGS
8,lispro,O,
9,",",O,
